Import Dependencies

In [121]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis

Read CSV and Convert to Pandas Dataframe
Drop Rows that have "Not assigned" in the Borough Column

In [223]:
df_data = pd.read_csv('Toronto_Data.csv')
df_data.drop(df_data.loc[df_data['Borough']=='Not assigned'].index, inplace=True)
df_data.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Replace Neighborhood's that are 'Not assigned' with thier corresponding Borough name, 
reset index and delete redundancy:
#### Note: it appears that 'Not assigned" has already been replaced in Wikipedia, see code below

In [225]:
df_data.loc[(df_data.Neighborhood == 'Not assigned'),'Neighborhood']= (df_data.Borough)
df_data = df_data.reset_index()
del df_data['index']
df_data

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


Combine Neighborhood's that have a common postal code
#### Note: it appears that this has already been done in Wikipedia

In [186]:
df2_data = df_data.groupby(['Postal Code', 'Borough'])['Neighborhood'].apply(lambda x: ','.join(set(x.dropna()))).reset_index()
# rename the column "PostalCode"
df2_data.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
df2_data.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [187]:
df2_data.shape

(103, 3)

Read CSV and Convert to Pandas Dataframe

In [217]:
geo_coor = pd.read_csv('https://cocl.us/Geospatial_data')

# rename the column "PostalCode"
geo_coor.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
geo_coor.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [226]:
df3_data = df2_data.merge(geo_coor, on='PostalCode', how='left')

In [227]:
df3_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Test Data:

In [229]:
# create a new test dataframe
column_names = ["PostalCode", "Borough", "Neighborhood", "Latitude", "Longitude"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(df3_data[df3_data["PostalCode"]==postcode], ignore_index=True)
    
test_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
1,M2H,North York,Hillcrest Village,43.803762,-79.363452
2,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
3,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
4,M4G,East York,Leaside,43.709060,-79.363452
5,M4M,East Toronto,Studio District,43.659526,-79.340923
6,M1R,Scarborough,"Wexford, Maryvale",43.750072,-79.295849
7,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437
8,M9L,North York,Humber Summit,43.756303,-79.565963
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420


Import Libraries for Part 3:

In [228]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

Create a map of Toronto, CA and superimpose the neighborhoods:

In [230]:
address = 'Toronto'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df3_data['Latitude'], df3_data['Longitude'], df3_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Define foursquare credentials and version:

In [231]:
CLIENT_ID = 'Client ID Here' # your Foursquare ID
CLIENT_SECRET = 'Client Secret Here' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KAGKHSGLIRZSUQWXBIMIDW2DMVZRYGI01ENDAPNWICAJQEHS
CLIENT_SECRET:OW50DXZMY4MSVAQ5LDS4K21IEAC2SJUCFVLRX0TCUIZKO3KP


Clean out all boroughs without the word "Toronto" in the name:

In [232]:
# clean borough names that contain the word Toronto
borough_names = list(df3_data.Borough.unique())

borough_w_toronto = []

for x in borough_names:
    if "toronto" in x.lower():
        borough_w_toronto.append(x)
        
borough_w_toronto

['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto']

Create new Dataframe with only these Boroughs:

In [235]:
# create a new DataFrame with only boroughs that contain the word Toronto
df4_data = df3_data[df3_data['Borough'].isin(borough_w_toronto)].reset_index(drop=True)
print(df4_data.shape)
df4_data.head()

(39, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


Create a map of these neighborhoods:

In [236]:
# create map of Toronto neighborhoods using latitude and longitude values
map_toronto2 = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df4_data['Latitude'], df4_data['Longitude'], df4_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto2)  
    
map_toronto2

Now, let's get the top 100 venues that are within a radius of 500 meters.

In [237]:
radius = 500
LIMIT = 100

venues = []

for lat, long, post, borough, neighborhood in zip(df4_data['Latitude'], df4_data['Longitude'], df4_data['PostalCode'], df4_data['Borough'], df4_data['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

Create new Dataframe:

In [238]:
# create new df based on the data extracted from the query
df5_data = pd.DataFrame(venues)

# name the column names
df5_data.columns = ['PostalCode', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(df5_data.shape)
print('{} venues were returned by Foursquare.'.format(df5_data.shape[0]))

(1627, 9)
1627 venues were returned by Foursquare.


Let's check how many venues were returned for each:

In [239]:
df5_data.groupby(["PostalCode", "Borough", "Neighborhood"]).count()

,,,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
PostalCode,Borough,Neighborhood,,,,,,
M4E,East Toronto,The Beaches,4,4,4,4,4,4
M4K,East Toronto,"The Danforth West, Riverdale",43,43,43,43,43,43
M4L,East Toronto,"India Bazaar, The Beaches West",23,23,23,23,23,23
M4M,East Toronto,Studio District,40,40,40,40,40,40
M4N,Central Toronto,Lawrence Park,3,3,3,3,3,3
M4P,Central Toronto,Davisville North,8,8,8,8,8,8
M4R,Central Toronto,"North Toronto West, Lawrence Park",19,19,19,19,19,19
M4S,Central Toronto,Davisville,35,35,35,35,35,35
M4T,Central Toronto,"Moore Park, Summerhill East",1,1,1,1,1,1


Let's find out how many unique categories can be curated from all the returned venues

In [240]:
print('There are {} uniques categories.'.format(len(df5_data['VenueCategory'].unique())))

There are 240 uniques categories.


Analyze Each Neighborhood:

In [241]:
# one hot encoding
toronto_onehot = pd.get_dummies(df5_data[['VenueCategory']], prefix="", prefix_sep="")

# add postal, borough and neighborhood column back to dataframe
toronto_onehot['PostalCode'] = df5_data['PostalCode'] 
toronto_onehot['Borough'] = df5_data['Borough'] 
toronto_onehot['Neighborhoods'] = df5_data['Neighborhood'] 

# move postal, borough and neighborhood column to the first column
fixed_columns = list(toronto_onehot.columns[-3:]) + list(toronto_onehot.columns[:-3])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(1627, 243)


,PostalCode,Borough,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M4K,East Toronto,"The Danforth West, Riverdale",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.

In [209]:
toronto_onehot.shape

(1627, 243)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category:

In [242]:
toronto_grouped = toronto_onehot.groupby(["PostalCode", "Borough", "Neighborhoods"]).mean().reset_index()

print(toronto_grouped.shape)
toronto_grouped

(39, 243)


,PostalCode,Borough,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,M4E,East Toronto,The Beaches,0.0000,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,...,0.000000,0.250000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000
1,M4K,East Toronto,"The Danforth West, Riverdale",0.0000,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,...,0.000000,0.023256,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.023256
2,M4L,East Toronto,"India Bazaar, The Beaches West",0.0000,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000
3,M4M,East Toronto,Studio District,0.0000,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.025000,0.0000,0.000000,0.025000
4,M4N,Central Toronto,Lawrence Park,0.0000,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000
5,M4P,Central Toronto,Davisville North,0.0000,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000
6,M4R,Central Toronto,"North Toronto West, Lawrence Park",0.0000,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.052632
7,M4S,Central Toronto,Davisville,0.0000,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,...,0.028571,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000
8,M4T,Central Toronto,"Moore Park, Summerhill East",0.0000,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000
9,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",0.0000,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.058824,0.000000,0.0000,0.000000,0.000000


Let's confirm the new size:

In [211]:
toronto_grouped.shape

(39, 243)

Let's create a new Pandas DataFrame along with the top 10 most common venues:

In [243]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ['PostalCode', 'Borough', 'Neighborhoods']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode'] = toronto_grouped['PostalCode']
neighborhoods_venues_sorted['Borough'] = toronto_grouped['Borough']
neighborhoods_venues_sorted['Neighborhoods'] = toronto_grouped['Neighborhoods']

for ind in np.arange(toronto_grouped.shape[0]):
    row_categories = toronto_grouped.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

# neighborhoods_venues_sorted.sort_values(freqColumns, inplace=True)
print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted

(39, 13)


,PostalCode,Borough,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,Neighborhood,Health Food Store,Pub,Trail,Yoga Studio,Dog Run,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
1,M4K,East Toronto,"The Danforth West, Riverdale",Greek Restaurant,Italian Restaurant,Coffee Shop,Furniture / Home Store,Bookstore,Frozen Yogurt Shop,Ice Cream Shop,Yoga Studio,Spa,Japanese Restaurant
2,M4L,East Toronto,"India Bazaar, The Beaches West",Park,Fast Food Restaurant,Pub,Light Rail Station,Sandwich Place,Burrito Place,Restaurant,Italian Restaurant,Intersection,Steakhouse
3,M4M,East Toronto,Studio District,Café,Coffee Shop,American Restaurant,Bakery,Brewery,Gastropub,Yoga Studio,Fish Market,Pet Store,Park
4,M4N,Central Toronto,Lawrence Park,Park,Swim School,Bus Line,Dim Sum Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
5,M4P,Central Toronto,Davisville North,Hotel,Gym,Department Store,Sandwich Place,Breakfast Spot,Food & Drink Shop,Park,General Travel,Greek Restaurant,Eastern European Restaurant
6,M4R,Central Toronto,"North Toronto West, Lawrence Park",Clothing Store,Coffee Shop,Yoga Studio,Sporting Goods Shop,Café,Chinese Restaurant,Diner,Fast Food Restaurant,Health & Beauty Service,Metro Station
7,M4S,Central Toronto,Davisville,Pizza Place,Sandwich Place,Dessert Shop,Sushi Restaurant,Café,Italian Restaurant,Coffee Shop,Gym,Farmers Market,Japanese Restaurant
8,M4T,Central Toronto,"Moore Park, Summerhill East",Tennis Court,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
9,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",Pub,Coffee Shop,Fried Chicken Joint,Skating Rink,Liquor Store,Restaurant,Sports Bar,Bank,Supermarket,Sushi Restaurant


Run k-means to cluster the Toronto areas into 5 clusters.

In [244]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop(["PostalCode", "Borough", "Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([4, 0, 0, 0, 2, 0, 0, 0, 3, 0], dtype=int32)

In [255]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
toronto_merged = df4_data.copy()

# add clustering labels
toronto_merged.insert(0, 'Cluster Labels', kmeans.labels_)

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.drop(["Borough", "Neighborhoods"], 1).set_index("PostalCode"), on="PostalCode")

print(toronto_merged.shape)
toronto_merged.head() # check the last columns!

(39, 16)


,Cluster Labels,PostalCode,Borough,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,4,M4E,East Toronto,The Beaches,43.676357,-79.293031,Neighborhood,Health Food Store,Pub,Trail,Yoga Studio,Dog Run,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
1,0,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,Greek Restaurant,Italian Restaurant,Coffee Shop,Furniture / Home Store,Bookstore,Frozen Yogurt Shop,Ice Cream Shop,Yoga Studio,Spa,Japanese Restaurant
2,0,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,Park,Fast Food Restaurant,Pub,Light Rail Station,Sandwich Place,Burrito Place,Restaurant,Italian Restaurant,Intersection,Steakhouse
3,0,M4M,East Toronto,Studio District,43.659526,-79.340923,Café,Coffee Shop,American Restaurant,Bakery,Brewery,Gastropub,Yoga Studio,Fish Market,Pet Store,Park
4,2,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,Park,Swim School,Bus Line,Dim Sum Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


In [256]:
# sort the results by Cluster Labels
print(toronto_merged.shape)
toronto_merged.sort_values(["Cluster Labels"], inplace=True)
toronto_merged

(39, 16)


,Cluster Labels,PostalCode,Borough,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,0,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752,Coffee Shop,Aquarium,Hotel,Café,Restaurant,Scenic Lookout,Sporting Goods Shop,Brewery,Fried Chicken Joint,Park
21,0,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817,Coffee Shop,Café,Restaurant,Hotel,Gym,American Restaurant,Japanese Restaurant,Italian Restaurant,Seafood Restaurant,Deli / Bodega
23,0,M5P,Central Toronto,"Forest Hill North & West, Forest Hill Road Park",43.696948,-79.411307,Park,Jewelry Store,Trail,Sushi Restaurant,Dessert Shop,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
24,0,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678,Café,Sandwich Place,Coffee Shop,Pub,Pizza Place,BBQ Joint,Liquor Store,History Museum,Indian Restaurant,Middle Eastern Restaurant
25,0,M5S,Downtown Toronto,"University of Toronto, Harbord",43.662696,-79.400049,Café,Restaurant,Bar,Italian Restaurant,Japanese Restaurant,Bookstore,Bakery,Dessert Shop,Pub,Chinese Restaurant
26,0,M5T,Downtown Toronto,"Kensington Market, Chinatown, Grange Park",43.653206,-79.400049,Café,Coffee Shop,Bakery,Vietnamese Restaurant,Mexican Restaurant,Vegetarian / Vegan Restaurant,Pizza Place,Bar,Gaming Cafe,Grocery Store
27,0,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420,Airport Service,Airport Lounge,Boutique,Harbor / Marina,Coffee Shop,Boat or Ferry,Rental Car Location,Sculpture Garden,Airport Terminal,Airport Gate
28,0,M5W,Downtown Toronto,Stn A PO Boxes,43.646435,-79.374846,Coffee Shop,Café,Italian Restaurant,Seafood Restaurant,Cocktail Bar,Beer Bar,Hotel,Restaurant,Japanese Restaurant,Lounge
29,0,M5X,Downtown Toronto,"First Canadian Place, Underground city",43.648429,-79.382280,Coffee Shop,Café,Restaurant,Hotel,Gym,Salad Place,Seafood Restaurant,Steakhouse,American Restaurant,Asian Restaurant
30,0,M6G,Downtown Toronto,Christie,43.669542,-79.422564,Grocery Store,Café,Park,Restaurant,Candy Store,Baby Store,Diner,Italian Restaurant,Coffee Shop,Nightclub


In [ ]:
Visualize Clusters

In [257]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, post, bor, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['PostalCode'], toronto_merged['Borough'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Clusters:

## Cluster-1

In [258]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,PostalCode,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,M5J,-79.381752,Coffee Shop,Aquarium,Hotel,Café,Restaurant,Scenic Lookout,Sporting Goods Shop,Brewery,Fried Chicken Joint,Park
21,M5L,-79.379817,Coffee Shop,Café,Restaurant,Hotel,Gym,American Restaurant,Japanese Restaurant,Italian Restaurant,Seafood Restaurant,Deli / Bodega
23,M5P,-79.411307,Park,Jewelry Store,Trail,Sushi Restaurant,Dessert Shop,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
24,M5R,-79.405678,Café,Sandwich Place,Coffee Shop,Pub,Pizza Place,BBQ Joint,Liquor Store,History Museum,Indian Restaurant,Middle Eastern Restaurant
25,M5S,-79.400049,Café,Restaurant,Bar,Italian Restaurant,Japanese Restaurant,Bookstore,Bakery,Dessert Shop,Pub,Chinese Restaurant
26,M5T,-79.400049,Café,Coffee Shop,Bakery,Vietnamese Restaurant,Mexican Restaurant,Vegetarian / Vegan Restaurant,Pizza Place,Bar,Gaming Cafe,Grocery Store
27,M5V,-79.394420,Airport Service,Airport Lounge,Boutique,Harbor / Marina,Coffee Shop,Boat or Ferry,Rental Car Location,Sculpture Garden,Airport Terminal,Airport Gate
28,M5W,-79.374846,Coffee Shop,Café,Italian Restaurant,Seafood Restaurant,Cocktail Bar,Beer Bar,Hotel,Restaurant,Japanese Restaurant,Lounge
29,M5X,-79.382280,Coffee Shop,Café,Restaurant,Hotel,Gym,Salad Place,Seafood Restaurant,Steakhouse,American Restaurant,Asian Restaurant
30,M6G,-79.422564,Grocery Store,Café,Park,Restaurant,Candy Store,Baby Store,Diner,Italian Restaurant,Coffee Shop,Nightclub


## Cluster-2

In [259]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,PostalCode,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,M5N,-79.416936,Home Service,Music Venue,Garden,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


## Cluster-3

In [260]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,PostalCode,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,M4N,-79.388790,Park,Swim School,Bus Line,Dim Sum Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
10,M4W,-79.377529,Park,Playground,Trail,Department Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


## Cluster-4

In [261]:

toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,PostalCode,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,M4T,-79.38316,Tennis Court,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


## Cluster-5

In [262]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,PostalCode,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,-79.293031,Neighborhood,Health Food Store,Pub,Trail,Yoga Studio,Dog Run,Dim Sum Restaurant,Diner,Discount Store,Distribution Center


Most of the locations appear in cluster 1.  Cluster 1 would appear to be a people dense group as most of the locations are food and shopping.